# 2D Design Template

# Overview

The purpose of this project is for you to apply what you have learnt in this course. This includes working with data and visualizing it, create model of linear regression or logistic regression, as well as using metrics to measure the accuracy of your model. 

Please find the project handout description in the following [link](https://sutdapac-my.sharepoint.com/:b:/g/personal/franklin_anariba_sutd_edu_sg/EaaE7XKJ0ZtJntc2IxEPiIYBVjijsZ5tUaPaH7mejSveWQ?e=1qADfd).


## Deliverables

You need to submit this Jupyter notebook together with the dataset into Vocareum. Use the template in this notebook to work on this project.

### Overview About the Problem

Describe here the problem you are trying to solve.

## Students Submission

Student names and contributions:
- Lee Jun Hui Ryan (1006652): Class implementation and evaluation
- Yee Jia Zhen (1006969): Data cleaning & website
- Lee Wei Jie (1006919): Data cleaning and video editing
- Aishwarya Shivakumar Iyer (1007141): Visualization code & website 
- Dione Hannah Woon (1007071): Class implementation and code documentation 

In [47]:
# All imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Dataset
Data Cleaning

Resources:
| Dataset                                                      | Source                     |
|--------------------------------------------------------------|----------------------------|
|Cost of a healthy diet (PPP dollar per person per day) Dataset|https://www.fao.org/faostat/en/#data/CAHD  |
Crude Birth Rates Dataset	                                   |https://data.worldbank.org/indicator/SP.DYN.CBRT.IN |
Food Price Inflation	                                       |https://www.kaggle.com/datasets/anshtanwar/monthly-food-price-estimates |
Global Peace Index	                                           |https://www.kaggle.com/datasets/ddosad/global-peace-index-2023
Global Terrorism Index                                         |https://www.kaggle.com/datasets/ddosad/global-terrorism-index-2023
Happiness Index	                                               |https://www.kaggle.com/datasets/anas123siddiqui/happiness-index-data
Human Development Index	                                       |https://www.kaggle.com/datasets/iamsouravbanerjee/human-development-index-dataset
Poverty Headcount	                                           |https://data.worldbank.org/indicator/SI.POV.NAHC?end=2022&start=2022&view=bar
Agriculture Gross Production Index/ Meat Gross Production Index|https://data.worldbank.org/indicator/AG.PRD.FOOD.XD?end=2021&start=1961&view=chart
World Freedom Index	                                           |https://www.kaggle.com/datasets/sujaykapadnis/world-freedom-index
Share of Employment in Agriculture	                           |https://www.fao.org/faostat/en/#data/OEA
World Population	                                           |https://www.fao.org/faostat/en/#data/OA
Others(QOL,PPI,SI,HCI,COLI.PPTIR,TCTI,PI,CI)                   |https://www.kaggle.com/datasets/ramjasmaurya/indexes-20122021

    


#### Step 1: Data Formatting  
We need to merge our datasets before we can use it for our multi linear regression model. Before we merge the datasets, we have to reformat them into our desired structure. Since most of our datasets are retrieved from FAOSTAT, the datasets share similar structures.  

We identified 2 patterns in our datasets and we name them **Type 1** and **Type 2**.

**Type 1**: 
- Country is represented by 'Country Name'.
- The years are the column names of the dataset.  

e.g.

| Country Name | Indicator         | 2008 | 2009 | 2010 | 2011 |
|--------------|-------------------|------|------|------|------|
| Afghanistan  | name of indicator | a    | b    | c    | d    |
| Bulgaria     | name of indicator | e    | f    | g    | h    |
| Colombia     | name of indicator | i    | j    | k    | l    |

<br>

**Type 2**:
- Country is represented by 'Country' or 'Area'.
- The data is represented by 'Value'.
- There is a 'Year' column for the years and the data is sorted in chronological order.

e.g.

| Country/Area | Indicator         | Year | Value |
|--------------|-------------------|------|-------|
| Afghanistan  | name of indicator | 2008 | a     |
| Bulgaria     | name of indicator | 2008 | b     |
| Afghanistan  | name of indicator | 2009 | c     |
| Bulgaria     | name of indicator | 2009 | d     |
| Afghanistan  | name of indicator | 2010 | e     |
| Bulgaria     | name of indicator | 2010 | f     |

<br>  

**Type 3**:
- Data is already in expected column (features), so we only need to remove rows with missing values via drop.
- The format of type 3 datasets aka 'indexes by year' is in the following format:

e.g. 

| Country | Year | Indicator Name | 
|---------|------|----------------|


**Expected Output**

Below is an illustration of how we would like to format our datasets. We want to keep the columns for the countries, years, and most importantly the values for their respective indicators. We will then create a new column with the indicator name as the column's name to store these values.  

| Country | Year | Indicator Name | 
|---------|------|----------------|

Since we have already decided on the year range (2017-2020), we will only extract data for these years.

e.g.

| Country     | Year | Human Development Index |  
|-------------|------|-------------------------|
| Afghanistan | 2017 | value1                  |
| Afghanistan | 2018 | value2                  |
| Afghanistan | 2019 | value3                  |
| Afghanistan | 2020 | value4                  | 



In [48]:
def country_ls(df):
    """Get list of countries in dataset.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataset.
        
    Returns
    -------
    country_ls : list
        List of countries from dataset.
    """
    
    country_ls = []

    for country in df['Country']:
        if country not in country_ls:
            country_ls.append(country)
    return country_ls


def format_type1(csv, column_name:str, name='Country'):
    """Format Type 1 datasets.

    Parameters
    ----------
    csv : pd.DataFrame
        Input DataFrame representing a Type 1 dataset.
    column_name : str
        Name of the indicator column.
    name : str
        Name of the column representing the country.

    Returns
    -------
    df_final : pd.DataFrame 
        Formatted DataFrame with columns: 'Country', 'Year', and the specified indicator column.
    """
    
    valid_columns = ['Country Name', '2017', '2018', '2019', '2020']
   
    for y in csv.keys():
        if y not in valid_columns:
            csv = csv.drop([y], axis=1)

    csv_T = csv.T
    srs_ls = []
    for index in range(csv_T.shape[1]):
        srs_ls.append(csv_T[index])

    df_ls = []
    for i in range(len(srs_ls)):
        srs = srs_ls[i]
        df = srs.to_frame()
        new_df = df.drop(index='Country Name', axis=1)
        country = srs['Country Name']
        new_df.insert(loc=0,
              column= name,
              value=country)
        new_df.rename(columns={i: column_name}, inplace=True)
        df_ls.append(new_df)
    df_final = pd.concat(df_ls).reset_index(drop=False)

    ls = country_ls(df_final)
    for c in ls:
        if df_final.loc[df_final['Country'] == c].shape[0] != 4: # 4 years
            df_final = df_final.drop(df_final[df_final['Country'] == c].index)
    print(type(df_final))
    df_final.rename(columns={"index": "Year"}, inplace=True)

    return df_final


def format_type2(csv, column_name:str):
    """Format Type 2 datasets.

    Parameters
    ----------
    csv : pd.DataFrame 
        Input DataFrame representing a Type 2 dataset.
    column_name : str 
        Name of the indicator column.

    Returns
    -------
    csv : pd.DataFrame
        Formatted DataFrame with columns: 'Country', 'Year', and the specified indicator column.
    """

    valid_columns = ['Area', 'Year', 'Value', 'Country']
    valid_years = [2017, 2018, 2019, 2020, '2017', '2018', '2019', '2020']
    
    for key in csv.keys():
        if key not in valid_columns:
            csv = csv.drop([key], axis=1)

    for yr in csv['Year']:
        if yr not in valid_years:
            csv = csv.drop(csv[csv['Year'] == yr].index)
    
    csv.rename(columns={"Area": "Country", "Value": column_name}, inplace=True)

    ls = country_ls(csv)
    for c in ls:
        if csv.loc[csv['Country'] == c].shape[0] != 4: # 4 years
            csv =csv.drop(csv[csv['Country'] == c].index)

    return csv


def format_type3(csv):
    """Format Type 3 datasets. ONLY for dataset 'indexes by year'.

    Parameters
    ----------
    csv : pd.DataFrame 
        Input DataFrame representing a 'indexes by year' dataset.
    
    Returns
    -------
    csv : pd.DataFrame
        Formatted DataFrame with columns: 'Country', 'Year', and relevant indicator columns.
    """

    valid_years = [2017, 2018, 2019, 2020, '2017', '2018', '2019', '2020']
    for yr in csv['Year']:
        if yr not in valid_years:
            csv = csv.drop(csv[csv['Year'] == yr].index)

    ls = country_ls(csv)
    for c in ls:
        if csv.loc[csv['Country'] == c].shape[0] != 4: # 4 years
            csv =csv.drop(csv[csv['Country'] == c].index)
    return csv

#### Step 2: Data Extraction

Every country should have data for each of the features that we have identified. Since the list of countries in each dataset is not the same, we cannot compare the data and would have to remove them before merging. This is to make sure every country has data for each of the features for all the datasets that we will be using for the model.

We wrote 2 functions for this task. 
1. `common_countries()`:\
This function returns a list of countries that can be found in all the datasets.\
Input: list of datasets that we will be using\
Output: list of common countries

2. `rm_irrelevant_countries()`\
This function uses the `common_countries()` function to remove the common countries in all the datasets.\
Input: list of datasets to clean\
Output: list of cleaned dataset (i.e. only data for common countries remain)

In [49]:
def common_countries(dfs:list):
    """Find common countries among multiple datasets.

    Parameters
    ----------
    dfs : list
        List of DataFrames to find common countries.

    Returns
    -------
    list 
        Sorted list of common countries found in all input DataFrames.
    """

    common_ls = []
    for df in dfs:
        common_ls.append(country_ls(df))

    common_elements = set(common_ls[0])
    for lst in common_ls[1:]:
        common_elements = common_elements & set(lst)

    return sorted(list(common_elements))


def rm_irrelevant_countries(dfs: list, country_keep: list):
    """Remove irrelevant countries from datasets.

    Parameters
    ----------
    dfs : list 
        List of DataFrames to be cleaned.
    country_keep : list
        List of countries to keep in the datasets.

    Returns:
    final_ls : list
        List of cleaned DataFrames with only relevant countries.
    """
        
    final_ls = []
    for csv in dfs:
        relevant_rows = csv['Country'].isin(country_keep)
        csv = csv[relevant_rows]
        final_ls.append(csv)
        
    return final_ls


#### Step 3: Merging

Now that the datasets are cleaned, all of them should have data of the same list of countries. The next step is to merge the datasets and remove all rows containing null values.

This can be done using `sort_dataframes_by_country()`.\
Input: list of dataframes to merge\
Output: merged dataframe

In [50]:
def sort_dataframes_by_country(dfs: list):
    """Sort and merge cleaned datasets by the 'Country' column, in alphabetical order.

    Parameters
    ----------
    dfs : list
        List of cleaned DataFrames to be sorted and merged.

    Returns
    -------
    clean_merged : pd.DataFrame
        Merged DataFrame with sorted rows based on the 'Country' column, in alphabetical order.
    """
    
    sorted_dfs = []
    for csv in dfs:
        sorted_df = csv.sort_values(by='Country')
        sorted_dfs.append(sorted_df.reset_index(drop=True))

    merged = pd.concat(sorted_dfs, axis=1 )
    clean_merged = merged.loc[:, ~merged.columns.duplicated(keep='first')]
    
    return clean_merged

In [51]:
# Read CSV and identify their types


# Set the path to the directory containing your CSV files
csv_files_path = 'datasets/*.csv'

# Get a list of all CSV files in the specified directory
csv_files = glob.glob(csv_files_path)

dataframes = {}

for i, file_path in enumerate(csv_files):
    df = pd.read_csv(file_path)

    if 'Value' in df.keys():
        print(f'{i}: type2 -> {file_path}')
    elif '2017' in df.keys():
        print(f'{i}: type1 -> {file_path}')
    else:
        print(f'{i}: type3 -> {file_path}')
        # print(df.keys())

    # Store the DataFrame in the dictionary with the variable name as the key
    dataframes[i] = df

0: type2 -> datasets/Cost of a healthy diet (PPP dollar per person per day).csv
1: type2 -> datasets/WorldPopulation - Employment.csv
2: type2 -> datasets/World Freedom Index .csv
3: type2 -> datasets/total_agriculture_production.csv
4: type2 -> datasets/total_meat_production.csv
5: type3 -> datasets/Indexes by Year.csv
6: type1 -> datasets/Poverty headcount.csv
7: type1 -> datasets/Human Development Index - Full.csv
8: type1 -> datasets/Global Terrorism Index 2023.csv
9: type2 -> datasets/Global Peace Index .csv
10: type2 -> datasets/Food Price Inflation.csv
11: type1 -> datasets/Crude BirthRates.csv
12: type2 -> datasets/WorldPopulation.csv
13: type1 -> datasets/happiness dataset.csv


In [52]:
# Data Formatting

df0 = format_type2(dataframes[0], 'Cost of a healthy diet (PPP dollar per person per day)')
df1 = format_type2(dataframes[1].loc[dataframes[1]['Indicator'].isin(['Employment by status of employment, total, rural areas'])], 'Employment')
df2 = format_type2(dataframes[2], 'World Freedom Index')
df3 = format_type2(dataframes[3], 'Agriculture Gross Production Index')
df4 = format_type2(dataframes[4], 'Meat Gross Production Index')
df5 = format_type3(dataframes[5])
df6 = format_type1(dataframes[6], 'Poverty Headcount')
df7 = format_type1(dataframes[7], 'Human Development Index')
df8 = format_type1(dataframes[8], 'Global Terrorism Index')
df9 = format_type2(dataframes[9], 'Global Peace Index')
df10 = format_type2(dataframes[10], 'Food Price Inflation')
df11 = format_type1(dataframes[11], 'Crude Birth Rates')
df12 = format_type2(dataframes[12], 'World Population')
df13 = format_type1(dataframes[13], 'Happiness Index')

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [53]:
# Please put all datasets into a list called df_ls
df_ls = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]
ck = common_countries(df_ls)
df_final = rm_irrelevant_countries(df_ls, ck)
df = sort_dataframes_by_country(df_final)

# For checking purposes
print(df.isnull().sum())
df = df.dropna() # drop all rows with missing data
display(df)

Country                                                    0
Year                                                       0
Cost of a healthy diet (PPP dollar per person per day)     0
Employment                                                 0
World Freedom Index                                        0
Agriculture Gross Production Index                         0
Meat Gross Production Index                                0
Quality of Life Index                                      0
Purchasing Power Index                                     0
Safety Index                                               0
Health Care Index                                          0
Cost of Living Index                                       0
Property Price to Income Ratio                             0
Traffic Commute Time Index                                 0
Pollution Index                                            0
Climate Index                                              0
Poverty Headcount       

,Country,Year,Cost of a healthy diet (PPP dollar per person per day),Employment,World Freedom Index,Agriculture Gross Production Index,Meat Gross Production Index,Quality of Life Index,Purchasing Power Index,Safety Index,...,Pollution Index,Climate Index,Poverty Headcount,Human Development Index,Global Terrorism Index,Global Peace Index,Food Price Inflation,Crude Birth Rates,World Population,Happiness Index
4,Austria,2017,2.772,1695.65,95,98.48,98.58,190.37,95.66,80.75,...,21.90,62.13,14.3,0.916,2.564,1.309,3.168317,10.0,8797496,7.006
5,Austria,2018,2.848,1705.07,94,100.47,96.98,182.50,82.38,76.27,...,22.19,77.3,13.3,0.917,2.342,1.272,0.863724,9.7,8840513,7.294
6,Austria,2019,2.915,1719.75,93,99.94,96.90,190.22,98.69,79.59,...,22.87,77.74,13.9,0.919,1.798,1.259,0.475737,9.6,8879940,7.139
7,Austria,2020,3.004,1715.81,93,101.82,95.66,191.05,96.70,78.63,...,21.97,77.74,14.7,0.913,3.803,1.265,2.775852,9.4,8907777,7.246
9,Belgium,2018,2.962,705.03,95,98.28,105.04,164.00,98.91,57.83,...,48.92,86.14,14.8,0.933,4.779,1.55,1.880804,10.4,11448595,6.923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Switzerland,2020,2.639,789.71,96,99.03,100.42,190.81,126.15,78.24,...,23.01,79.78,16.0,0.959,3.094,1.365,0.378575,10.3,8638613,7.494
120,Thailand,2017,3.971,20396.20,32,102.33,99.80,101.18,38.59,52.75,...,72.86,68.64,7.9,0.79,6.901,2.189,0.025742,9.98,70898202,5.999
121,Thailand,2018,4.042,20659.06,31,105.29,100.52,103.26,40.86,53.34,...,72.21,69.45,9.9,0.795,6.662,2.147,1.039929,9.662,71127802,6.424
122,Thailand,2019,4.181,20443.25,30,102.26,101.42,101.88,35.45,59.52,...,75.07,69.45,6.2,0.804,6.567,2.181,1.505000,9.377,71307763,6.072


In [ ]:
# Export data

df.to_csv('merged_dataset.csv', index=False)

### Features and Target Preparation

*Describe here what are the features you use and why these features. Put any Python codes to prepare and clean up your features. Do the same thing for the target. Describe your target and put any codes to prepare your target.*

In this section, we will be visualizing our data and deciding which features to use for our model. Our target is "Cost of a healthy diet (PPP dollar per person per day)" in order for our model to address our problem statement. The data also needs to be prepared in other ways (e.g. adding intercept term, normalization, type conversions, etc) will be carried out by our multiple linear regression model (implemented as a class). Please refer to the section "Building Model" for more details. 

In [54]:
df = pd.read_csv("merged_dataset.csv") # Load in cleaned data
target = "Cost of a healthy diet (PPP dollar per person per day)" # Chosen target
features = list(df.columns) # all variables in DataFrame

# Remove non-features:
features.remove(target)
features.remove("Year")
features.remove("Country")
print(features)

['Employment', 'World Freedom Index', 'Agriculture Gross Production Index', 'Meat Gross Production Index', 'Quality of Life Index', 'Purchasing Power Index', 'Safety Index', 'Health Care Index', 'Cost of Living Index', 'Property Price to Income Ratio', 'Traffic Commute Time Index', 'Pollution Index', 'Climate Index', 'Poverty Headcount', 'Human Development Index', 'Global Terrorism Index', 'Global Peace Index', 'Food Price Inflation', 'Crude Birth Rates', 'World Population', 'Happiness Index']


Let's visualize our data by plotting each feature against our target variable.

In [ ]:
# Prepare data for plots (everything should be 1-dimensional)
num_features = len(features)
y = df[target] # Series

# Calculate the number of rows and columns for subplots (n plots per row)
n, size = 3, 5 # configuration parameters for visualization
num_rows = int(np.ceil(num_features / n))
num_cols = min(num_features, n)
width = size * num_cols
height = size * num_rows

# Set up subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(width, height))
if num_features > 1:
    axes = axes.flatten()
else:
    axes = [axes]

features_r2 = [] # store features and their corresponding r2 values

# Plot target against each feature
for i in range(num_features):
    feature = features[i]
    x = df[feature] # Series
    r = np.corrcoef(x, y)[0, 1] # correlation coefficient
    r2 = np.square(r)
    features_r2.append((feature, r2))
    
    sns.scatterplot(x=x, y=y, ax=axes[i], color="blue", marker="o", label=f"r2 = {r2:.5f}")

    # Add labels and title
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel(target)
    axes[i].set_title("  ")
    # axes[i].set_title(f"Scatter plot of {target} against {feature}")
    axes[i].legend()

# Hide remaining empty subplots
for j in range(i+1, len(axes)):
    axes[j].set_axis_off()

plt.tight_layout()
plt.show()

In the code above, we have plotted out scatter plots of our target variable against each feature and calculated the respective r2 values. The closer r2 is a value between 0 and 1; the larger the r2 value, the stronger the correlation between the variables. We have stored the features and their r2 values in a list called 'features_r2'.<br>For multiple linear regression, we want to limit the number of features we use and only keep the most relevant features. This is because we have limited data points and we want to avoid the 'curse of dimensionality'. Having 8-10 times more data points than features should be good enough. As such, we will only be using 8 of the most relevant features, where relevance is decided based on the r2 value of the feature and target, which indicates the strength of their correlation. We will sort the features_r2 list in descending order of r2 value and use the first 8 features.

In [ ]:
# Sort features by their r2 value in descending order:
features_r2.sort(reverse=True, key=lambda x: x[1])

# Display the 8 features with the largest r2 (strongest correlation with target):
chosen_features = [] 
for i in range(8):
    feature = features_r2[i][0]
    r2 = features_r2[i][1]
    print(f"{feature} : {r2:.5f}")
    chosen_features.append(feature)

Our chosen target and features have been stored in the 'target' and 'chosen_features' variables respectively. Of course, for multiple linear regression, this data will need to be prepared properly (e.g. adding intercept term, normalization, storing in NumPy arrays, etc) to be used by training and prediction algorithms. For this project, we have decided to use a class based approach where these preparations are performed by the model - the user only needs to prepare the appropriate pandas DataFrame. More details on this in the section below.

### Building Model

*Describe your model. Is this Linear Regression or Logistic Regression? Put any other details about the model. Put the codes to build your model.*

We implemented multiple linear regression using a class, in which an instance represents an individual multiple linear regression model. We used a class for modularity and reusability, allowing us to easily use the model in different parts of the code and in other projects. Additionally, a class provides a layer of abstraction that allows users of the class to work with the model without understanding the internal workings and implementation details. Furthermore, because things like parameter values are stored in the instance itself, it is easy to experiment with models that have different configurations. Finally, using a class allows us to easily implement variations of the model using inheritance (as we will demonstrate later).

We have designed this class such that each model can be trained to predict multiple target variables with the same input features. In other words, the model stores multiple sets of parameter values for the prediction of different target variables. Additionally, as mentioned in the 'Features and Target Preparation' section, our class handles data preparation (type conversion, intercept term, normalization, etc) for the user. Please see the class and method documentation below for more details.

In [ ]:
class MultipleLinearRegression:
    """Class for a multiple linear regression model.

    Extended Summary
    ----------------
    The model is designed to be trained to predict multiple target variables using
    the same input features Hence, the model stores separate sets of parameter values
    for the prediction of each target variable.
    General user flow:
    1) Store data (expects pandas DataFrames) using store_data().
    2) Train the model to predict the specified target variable using train().
    3) Validate the model's performance using validate().

    Attributes
    ----------
    params : dict
        Dictionary that stores the different sets of parameter values for different
        target variables. The key is the name of a target variable (str) while
        the value is a numpy.ndarray containing the parameter values. New key-value
        pairs (i.e. new parameters) are added or updated when the model is trained to
        predict a specified target variable. Each numpy.ndarray (set of parameter values)
        has a shape of (n+1,1) where n is the number of features.
    features, targets : list of str
        A list of strings containing the column names of the features and targets respectively.
        Initialised when data is stored in the instance using store_data().
    train_features, test_features, train_targets, test_targets : pandas.DataFrames
        Pandas Dataframes containing the features/targets for training/testing. Initialised
        when data is stored in the instance using store_data().
    means, stds : float
        The mean and standard deviation of the training features, initialised during training.
        Used for normalization of input features for model predictions.

    Methods
    -------
    store_data(data, feature_names, target_names, random_state=None, test_size=0.5)
        Takes the given data and separates the features and targets. Then splits the data
        for training and testing before storing it in the model's attributes.
    add_transformed_feature(feature_name, new_feature_name, transform_func, replace=False)
        Transforms a specified feature and adds it to the stored data.
    train(target, alpha=0.01, epochs=1000, retrain=False, convergence_threshold=1e-6, show=True)
        Trains the model to predict the specified target variable with the input features. Stores
        the parameters in the model.
    predict(df_features, target)
        Given some input data of features, the model returns its predictions for the specified target
        variable using linear regression (with its stored parameters).
    validate(target, visualize=True)
        Evaluate the model's ability to predict the specified target variable (its performance) using the
        test data. Will show visualizations of the predicted values against the true values for the test data
        and return various performance metrics.
    """

    def __init__(self):
        """Constructs a new instance of MultipleLinearRegression"""

        self.params = {} # dictionary of weights (numpy arrays)
        self.features, self.targets = None, None # list of feature/target column names
        self.train_features, self.train_targets = None, None
        self.test_features, self.test_targets = None, None
        self.means, self.stds = None, None # for normalization (initialised during training)


    def store_data(self, data, feature_names, target_names, random_state=None, test_size=0.5):
        """Prepares and stores data in the model (instance).

        Extended Summary
        ----------------
        Will split the dataset into two: one portion for training and the other for testing (model validation).
        The features and targets for training/testing are stored in the instance as attributes.

        Parameters
        ----------
        data : pandas.DataFrame
            DataFrame that contains all the relevant data (features and targets).
        feature_names, target_names : list
            Lists containing the column names for the features and targets respectively.
        random_state : int, optional
            Integer representing a seed for splitting of the data into training and test sets.
            Setting a seed to a specific value ensures reproducibility (default is None, which means
            that no seed will be specified and data splitting will be random).
        test_size : float, default 0.5
            Float value between 0 and 1 that represents the fraction of the data that will be in the test set.
        """

        # Split data into features and targets
        self.features = feature_names.copy()
        self.targets = target_names.copy()
        df_features = data.loc[:, feature_names]
        df_targets = data.loc[:, target_names]

        # Split data into training and testing sets, then store in instance
        self.train_features, self.test_features, self.train_targets, self.test_targets = self._split_data(df_features, df_targets, random_state, test_size)


    def add_transformed_feature(self, feature_name, new_feature_name, transform_func, replace=False):
        """Takes a feature in the stored data and stores a transformed version of it.

        Extended Summary
        ----------------
        Transforms the specified feature (already stored in the model) by applying the transformation
        function on each feature value. The new transformed feature values can be stored as a new
        feature or can replace the original feature which was transformed. Important to note: the model
        should be retrained after adding a transformed feature.

        Parameters
        ----------
        feature_name : str
            Column name of the feature to be transformed. The feature should exist in the model's
            stored data.
        new_feature_name : str
            The column name of the transformed feature to be added.
        transform_func:
            The name of the transformation function i.e. the function to be applied to the specified feature
            values to transform them.
        replace : bool, default False
            Determines whether transformed features will replace the original feature or be added as a separate
            feature. (Default is False, so the transformed features will be added as a new column with the
            column name specified with parameter new_feature_name).
        """

        if feature_name in self.features:
            # add new column of transformed features to both training and test features
            train_features_transformed = self.train_features[feature_name].apply(transform_func)
            test_features_transformed = self.test_features[feature_name].apply(transform_func)

            if replace == True:
                # Replace data:
                self.train_features[feature_name] = train_features_transformed
                self.test_features[feature_name] = test_features_transformed

                # Rename:
                self.train_features.rename(columns={feature_name: new_feature_name}, inplace=True)
                self.test_features.rename(columns={feature_name: new_feature_name}, inplace=True)
                self.features = [new_feature_name if item == feature_name else item for item in self.features]

            else:
                # Add as new column
                self.train_features[new_feature_name] = train_features_transformed
                self.test_features[new_feature_name] = test_features_transformed
                self.features.append(new_feature_name)
        else:
            raise ValueError(f"No feature called {feature_name} in stored data.")


    def train(self, target, alpha=0.01, epochs=1000, retrain=False, convergence_threshold=1e-6, show=True):
        """Train the model to predict the specified target variable (using the stored training data).

        Extended Summary
        ----------------
        Performs batch gradient descent with training data stored using the store_data method.
        Updates the 'param' attribute with new parameter values. Cost is defined as half the mean squared error.

        Parameters
        ----------
        target : str
            Column name of target variable to be predicted.
        alpha : float, default 0.01
            Learning rate for gradient descent.
        epochs : int, default 1000
            Number of gradient descent steps.
        retrain : bool, default False
            If True, reinitialize the model parameters. (Default is False, meaning
            training will start using current parameter values if they exist)
        convergence_threshold : float, default 1e-6
            Threshold for convergence based on the change in cost. Training will end
            prematurely if difference in cost between iterations is less than the threshold.
        show : bool, default True
            If True, will show change in cost during training.

        Returns
        -------
        self.params : numpy.ndarray
            NumPy array containing parameter values.
        J_storage : numpy.ndarray
            NumPy array containing the cost after each iteration of gradient descent.
        """

        # Check that there is data stored for training and validation:
        if any(data is None for data in (self.train_features, self.train_targets, self.test_features, self.test_targets)):
            raise ValueError("Data missing. Run the store_data() method first.")
        # Check that valid target name is specified:
        elif target not in self.targets:
            raise ValueError("Specified target not found in stored data.")


        # Normalize training features:
        train_features_norm, self.means, self.stds = self._normalize_z(self.train_features)

        # Prepare data in numpy arrays:
        X_train = self._prepare_X(train_features_norm)
        y_train = self._prepare_y(self.train_targets.loc[:, [target]])
        m, n = X_train.shape  # m = number of training examples, n = number of features (including intercept term)

        # Validate parameters
        params = self.params.setdefault(target, np.zeros((n,1)) ) # initialize if not exist
        if retrain or params.shape[0] != n: # check if dimensions match
            self.params[target] = np.zeros((n,1)) # reset

        # Gradient Descent
        J_storage = np.zeros(epochs)
        if show: print("TRAINING START")
        for i in range(epochs):
            y_hat = self._calc_linreg(X_train, self.params[target])
            err = y_hat - y_train

            # Measure cost J in each iteration (epoch):
            J = np.matmul(err.T, err) / (2*m)
            J_storage[i] = J[0][0] # scalar value of (1,1) array

            # Update parameters (according to partial derivatives of cost function J):
            deriv = np.matmul(X_train.T, err) / m
            self.params[target] = self.params[target] - alpha * deriv  # update stored parameters

            if show: print(f"Epoch {i + 1:<4} || Cost: {J[0][0]:.5f}")
            # Check for convergence:
            if i > 0 and abs(J_storage[i] - J_storage[i-1]) < convergence_threshold:
                print(f"Converged (threshold = {convergence_threshold}). Stopping training.")
                break

        if show:
            print("TRAINING END")
            plt.figure(figsize=(5, 3)) # size of plot
            sns.lineplot(x=range(1, len(J_storage) + 1), y=J_storage)
            plt.xlabel("Epochs")
            plt.ylabel("Cost (J)")
            plt.title("Training Cost Over Time")
            plt.show()

        return self.params, J_storage


    def predict(self, df_features, target):
        """Model returns predictions of specified target using the given input features.

        Parameters
        ----------
        df_features : pandas.DataFrame
            Data containing input features to predict with, where each row is a set of input features (x).
            It is a DataFrame with shape (m,n), where m is the number of data points to be predicted
            and n is the number of feature variables.
        target : str
            Column name of target variable to be predicted.

        Returns
        -------
        y_hat : numpy.ndarray
            NumPy array that has a shape of (m,1) and contains the model's predictions for each
            of the input data points.
        """

        self._target_valid(target)
        norm_features,_,_ = self._normalize_z(df_features, self.means, self.stds) # normalize input features using training feature means and stds
        X = self._prepare_X(norm_features)
        y_hat = self._calc_linreg(X, self.params[target])
        return y_hat


    def validate(self, target, visualize=True):
        """Evaluate the model's performance using testing data and return performance metrics.

        Extended Summary
        ----------------
        Using the testing data, the model will be evaluated by comparing its predictions (of the
        specified target variable) with the true target values. This comparison is visualised with
        scatter plots of the target variable against each input feature (optional). The following metrics,
        which evaluate the multiple linear regression model's predictions, are computed and returned:
        - Mean Square Error
        - Mean Absolute Error
        - R2 coefficient of determination
        - Adjusted R2 coefficient of determination

        Parameters
        ----------
        target : str
            Column name of target variable to be predicted. The model will be evaluated based on its
            predictions for this target variable.
        visualize : bool, default True
            If True, comparison between predictions and true values is visualized.

        Returns
        -------
        mse, mae, r2, r2_adj : float
            The numerical values of the model's performance metrics mentioned above.

        Notes
        -----
        - The R2 coefficient is usually a value between 0 and 1, in which 0 indicates that the model explains
        none of the variability and 1 indicates that the model explains all the variability. When there is more
        than 1 feature however, it is possible for R2 coefficient to be negative if the fit is particularly bad.
        - For multiple linear regression, the R2 coefficient is not a good performance metric, as increasing the
        number of features always increases the R2 value. The adjusted R2 value (r2_adj) is a metric that accounts
        for this "inflation" of the R2 value when the number of feature variables increases, and is therefore
        more appropriate for evaluating model fit.
        """

        self._target_valid(target)
        df_features = self.test_features
        df_targets = self.test_targets

        y_hat = self.predict(df_features, target)
        y = self._prepare_y(df_targets.loc[:, [target]])
        if visualize == True:
            self._visualize(df_features, df_targets, target, y_hat)
        mse, mae, r2, r2_adj = self._calc_metrics(y_hat, y)
        print(f"Mean Square Error: {mse:.5f} | Mean Absolute Error: {mae:.5f} | R2 Coefficient of Determination: {r2:.5f} | Adjusted R2: {r2_adj:.5f}")
        return mse, mae, r2, r2_adj


    """Helper (private) functions"""

    def _visualize(self, df_features, df_targets, target, y_hat, size=5, n=3):
        """Visualize comparison between model prediction and true target values with scatter plots.

        Parameters
        ----------
        df_features, df_targets : pandas.DataFrame
            Dataframes containing the features and targets.
        target : str
            Column name of target variable being predicted.
        y_hat : numpy.ndarray
            NumPy array containing model's predictions.
        size: int, default 5
            Determines the size of the scatterplots.
        n : int, default 3
            Maximum number of scatterplots in a row.
        """

        # Prepare data for plots (everything should be 1-dimensional)
        num_features = len(self.features)
        y = df_targets[target]
        y_hat = y_hat.flatten()

        # Calculate the number of rows and columns for subplots (n plots per row)
        num_rows = int(np.ceil(num_features / n))
        num_cols = min(num_features, n)
        width = size * num_cols
        height = size * num_rows

        # Set up subplots
        fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(width, height))
        if num_features > 1:
            axes = axes.flatten()
        else:
            axes = [axes]

        # Subplot style parameters
        y_params = {
            'color': 'blue',
            'marker': 'o',
            'label': "True values"
        }

        y_hat_params = {
            'color': 'orange',
            'marker': 'o',
            'label': "Predictions"
        }

        # Plot target against each feature
        for i in range(num_features):
            feature = self.features[i]
            x = df_features[feature]
            sns.scatterplot(x=x, y=y, ax=axes[i], **y_params)
            sns.scatterplot(x=x, y=y_hat, ax=axes[i], **y_hat_params)

            # Add labels and title
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel(target)
            axes[i].set_title("  ")
            # axes[i].set_title(f"Scatter plot of {target} against {feature}")
            axes[i].legend()

        # Hide remaining empty subplots
        for j in range(i+1, len(axes)):
            axes[j].set_axis_off()

        plt.tight_layout()
        plt.show()


    def _calc_metrics(self, y_hat, y):
        """Calculate 4 metrics for model evaluation.

        Parameters
        ----------
        y_hat, y : numpy.ndarray
            Arrays containing model's predictions (y_hat) and the true values (y)
            for m training examples. Both should be of shape (m,1).
        """

        r2 = self.R2(y_hat, y)
        r2_adj = self.adjusted_R2(r2, y.shape[0], len(self.features))
        mse = self.mean_squared_error(y_hat, y)
        mae = self.mean_abs_error(y_hat, y)
        return mse, mae, r2, r2_adj


    def _target_valid(self, target):
        """Checks whether given target name is valid (found in stored data)
        and whether the model has been trained to predict it yet."""

        if target not in self.targets:
            raise ValueError("Specified target not found in stored data.")
        elif target not in self.params.keys():
            raise ValueError("Model has not been trained to predict this target variable yet. Run the train() method.")


    @staticmethod
    def _calc_linreg(X, params):
        """Calculates linear regression equation.

        Parameters
        ----------
        X : numpy.ndarray
            Array of shape (m,n+1) where m is the number of data examples (rows)
            and n is the number of feature variables (excluding the intercept term).
        params : numpy.ndarray
            Array of shape (n+1,1) i.e. column vector where the ith value is the
            coefficient for the ith feature (the 0th coefficient is for intercept term)

        Returns
        -------
        np.ndarray
            Array of shape (m,1).
        """

        return np.matmul(X, params)


    @staticmethod
    def _split_data(df_features, df_targets, random_state, test_size):
        """Splits the given feature and target dataframes into a test
        and train set randomly.

        Parameters
        ----------
        df_features, df_targets : pandas.DataFrame
            DataFrames to be split into test and train sets.
        random_state : int
            Integer representing a seed for splitting of the data into training and test sets.
            Setting a seed to a specific value ensures reproducibility. If parameter is set to None
            instead of an integer, no seed will be specified and data splitting will be random.
        test_size : float
            Fraction (between 0 to 1) of the data to be allocated as the test set.

        Returns
        -------
        df_features_train, df_targets_train : pandas.DataFrame
            Features and targets for training set.
        df_features_test, df_targets_test : pandas.DataFrame
            Features and targets for test set.
        """

        # Set the random seed
        if random_state is not None:
          np.random.seed(random_state)

        indexes = df_features.index
        k = int(len(indexes) * test_size)

        # Randomly shuffle the indices to create a random test set
        test_indexes = np.random.choice(indexes,k,replace=False)
        train_indexes = list(set(indexes) - set(test_indexes))
        df_features_train = df_features.loc[train_indexes, :]
        df_features_test = df_features.loc[test_indexes, :]
        df_targets_train = df_targets.loc[train_indexes, :]
        df_targets_test = df_targets.loc[test_indexes, :]

        return df_features_train, df_features_test, df_targets_train, df_targets_test


    @staticmethod
    def _normalize_z(dfin, columns_means=None, columns_stds=None):
        """Normalize input dataframe using Z normalization

        Parameters
        ----------
        dfin : pandas.DataFrame
            Input data to be normalized.
        columns_means, columns_stds : float, optional
            Mean and standard deviation values used for normalization. If not
            specified (default None), the values will be computed using the input data.

        Returns
        -------
        dfout : pandas.DataFrame
            DataFrame containing normalized data.
        columns_means, columns_stds : float
            Will return the mean and standard deviation calculated using input DataFrame
            or the values passed in as arguments (depending on whether values for
            columns_means and columns_stds were passed in).
        """

        if columns_means is None:
            columns_means = dfin.mean(axis=0) # mean of all rows in each column
        if columns_stds is None:
            columns_stds = dfin.std(axis=0)

        dfout = (dfin - columns_means) / columns_stds # broadcasting
        return dfout, columns_means, columns_stds


    @staticmethod
    def _prepare_X(df_features):
        """Turns DataFrame of feature values into design matrix X.

        Extended Summary
        ----------------
        Takes in DataFrame with shape (m,n) where m is the number of data
        examples and n is the number of features. It will be converted into
        a 2-dimensional NumPy array and a column of 1s (x0) will be added for
        the intercept term.

        Parameters
        ----------
        df_features : pandas.DataFrame
            DataFrame of feature values with shape (m,n). (If a pandas.Series is passed
            in instead, it will be converted to pandas DataFrame automatically.)

        Returns
        -------
        X : numpy.ndarray
            NumPy array representing design matrix X with shape (m,n+1).
        """

        if type(df_features) == pd.Series:
            df_features = pd.DataFrame(df_features)

        rows, cols = df_features.shape
        if type(df_features) == pd.DataFrame:
            features = df_features.to_numpy() # Convert dataframe to np array
        else:
            features = df_features
        features = features.reshape(-1, cols) # ensure it is 2-dimensional array

        # Add new column of features x0 = 1 (intercept term):
        x0 = np.ones((rows,1))
        X = np.concatenate((x0, features), axis=1)
        return X # (m, n+1)


    @staticmethod
    def _prepare_y(df_target):
        """Turns DataFrame of target values into column vector y.

        Extended Summary
        ----------------
        Takes in DataFrame with shape (m,1) where m is the number of data
        examples. It will be converted into a 2-dimensional NumPy array with shape (m,1).

        Parameters
        ----------
        df_target : pandas.DataFrame
            DataFrame of feature values with shape (m,1). (If a pandas.Series is passed
            in instead, it will be converted to pandas DataFrame automatically.)

        Returns
        -------
        y : numpy.ndarray
            NumPy array representing column vector y with shape (m,1).
        """

        if type(df_target) == pd.Series:
            df_target = pd.DataFrame(df_target)

        cols = df_target.shape[1]
        if type(df_target) == pd.DataFrame:
            target = df_target.to_numpy()
        else:
            target = df_target

        target = target.reshape(-1, cols) # reinforce that targets should be 2-d np array
        return target


    @staticmethod
    def mean_squared_error(y_hat, y):
        """Compute Mean Squared Error (MSE)."""

        mse =  np.mean(np.square(y_hat - y))
        return mse

        '''# Alternatively, using matrices:
        m = y.shape[0] # number of data examples
        err = y_hat - y
        mse = np.matmul(err.T, err) / m # no need half factor
        mse = mse[0][0]
        # is matrix approach better? check which is computationally faster
        '''


    @staticmethod
    def R2(y_hat, y):
        """Compute R-squared value."""

        y_bar = np.mean(y)
        err = y_hat - y
        ss_residual = np.sum(np.square(y - y_hat))
        ss_total = np.sum(np.square(y - y_bar)) # y_bar is broadcasted
        r2 = 1 - (ss_residual / ss_total)
        # It is possible for r2 to be negative (ss_res > ss_total) if not model not well fitted.
        return r2

        '''# Alternatively, using matrices:
        # err = y_hat - y
        # ss_res = np.matmul(err.T, err)
        # ss_tot = np.matmul((y - y_bar).T, (y - y_bar))
        '''


    @staticmethod
    def adjusted_R2(r2, num_examples, num_features):
        """Compute Adjusted R-squared value."""

        return 1 - ( (1-r2)*(num_examples-1) ) / (num_examples-num_features-1)


    @staticmethod
    def mean_abs_error(y_hat, y):
        """Compute Mean Absolute Error (MAE)."""

        return np.mean(np.abs(y_hat - y))


#### Step 1: Prepare data for model
In the 'Features and Target Preparation' section, we have already prepared our pandas dataframe with all our data and chosen our feature and target variables. 

In [ ]:
print("Target:", target) # chosen target variable
print("Chosen features:", chosen_features) # chosen feature variables
df # entire DataFrame

#### Step 2: Prepare model
Instantiate Multiple Linear Regression model and load data into it. Indicate chosen target and feature variables. As noted in the class documentation, the model will split the dataset into a test set and training set.

In [ ]:
model1 = MultipleLinearRegression()
model1.store_data(df, chosen_features, [target], random_state=100, test_size=0.9)

#### Step 3: Train model
Note: Because the 'retrain' parameter of the train method is set to False by default, if you run the cell multiple times, the model will start training using its currently saved parameter values (obtained from the previous training). Hence, why the training cost curve is different each time you run the cell below.

In [ ]:
model1.train(target, alpha=0.1, epochs=25)
pass

### Evaluating the Model

*Describe your metrics and how you want to evaluate your model. Put any Python code to evaluate your model. Use plots to have a visual evaluation.*
<br>
Our class has a method called 'validate', in which the model will use the test set to evaluate its performance by comparing its predictions to the true target values. This method computes and returns four performance metrics that can be used to evaluate the model:
1) Mean Square Error (MSE): it is the average squared difference between the predicted and actual values. While MSE is useful for gradient descent as a cost function, it should be noted that MSE is extremely sensitive to outliers, as large deviations between predicted and actual values are made even larger when squared. Additionally, MSE does not provide any insight on the directionality of the deviation (i.e. whether the model underpredicts or overpredicts).
2) Mean Absolute Error (MAE): the average absolute difference between the predicted and actual values. MAE is less sensitive to outliers than MSE as the deviations are not squared but also similarly does not provide insight on directionality.
3) R2 coefficient of determination (not considered): a value *typically* between 0 and 1 that provides a measure of goodness of fit of the model as it is the proportion of variance in the dependent variable that is explained by the model. An R2 value of 1 indicates that the model perfectly explains the variance while 0 means the model does not explain any of the variance. **However**, in the case where more than 1 feature is considered, it is possible for R2 to be a negative value if the model has a particularly terrible fit. Furthermore, R2 is not a useful measure when there is more than 1 feature, as its value will with more features (predictors). Hence, we will not be considering R2 when evaluating our model.
4) **Adjusted** R2 coefficient of determination: the adjusted R2 coefficient is essentially the R2 coefficient except that it takes into account the number of features in the model. This makes it a more meaningful metric for evaluating our multiple linear regression model, in which there may be multiple features (and allows us to meaningfully compare models with different number of features). 
<br>
Generally, MSE and MAE are measures of the model's prediction accuracy, while adjusted R2 is a measure of model fit. 

In [ ]:
# Evaluate the model and visualize its accuracy
model1.validate(target)
pass

Unfortunately, as can be seen in the scatter plots, our model's predictions are very off. Our adjusted R2 is even negative, which can happen in the multiple linear regression case if the model is very badly fitted. We will be improving our model's performance in the following steps. 

### Improving the Model

"Discuss any steps you can do to improve the models. Put any python codes. You can repeat the steps above with the codes to show the improvement in the accuracy."

In this section, we will be improving our model's performance. We have already taken some steps towards this previously, by cleaning our data (e.g. removing outliers that are essentially noise) and choosing our features carefully. Now, we will be attempting to improve our model with the following approaches:
1) Hyperparameter Tuning
2) Feature transformation
3) Weighted Linear Regression

#### Approach 1: Hyperparameter Tuning
Thanks to our class-based implementation, we can create different models trained with varying configurations (different hyperparameter values) and compare model performance. The hyperparameters we can tune are:
- alpha: the learning rate, which determines the size of the step taken during gradient descent
- epochs: the number of iterations during training in gradient descent i.e. the number of steps taken
- test_size: the fraction of our data set used for testing (as opposed to training)

To avoid cluttering this file, we have conducted hyperparameter tuning elsewhere and have landed on the following hyperparameters values used in model2:

In [ ]:
model2 = MultipleLinearRegression()
model2.store_data(df, chosen_features, [target], random_state=100, test_size=0.5)
model2.train(target, alpha=0.01, epochs=600)
model2.validate(target)
pass

model1 hyperparameters: test_size = 0.9, alpha = 0.1, epochs = 25
<br>model2 hyperparameters: test_size = 0.5, alpha = 0.01, epochs = 600

As you can see from the results above, model2 is significantly more accurate in its predictions than model1. It has lower mean square error and mean absolute error. Its adjusted R2 coefficient is also significantly better. In our investigation on the hyperparameters, we have observed the following:
- **alpha** (learning rate): In general, a good value is somewhere between 0.1 and 0.01. Using smaller values means that more iterations are needed to converge to the cost function minima and therefore training would require more time (which is a non-issue for our small-scale project). If the learning rate is too high (e.g. 1), it is possible for the cost to increase during training instead. This happens because the gradient descent step becomes too big and the parameter values overshoot past the minima, causing it to diverge from rather than converge to minima. As a future improvement to our model, learning rate scheduling can be implemented, in which the learning rate starts off large (for quicker convergence) but gets much smaller when the parameters are closer to the minima (to avoid divergence with exploding gradients).
- **epochs**: in general, the higher the better - the more iterations of gradient descent, the lower the cost function gets. However, as can be seen in the cost function plot above, the rate at which the cost decreases diminishes significantly after a certain point and therefore there is a trade-off between time and accuracy.
- **test_size**: if too much of the data set is allocated to the test set, there may be insufficient data for training, which may result in the model being unable to pick up on the general trends in the overall data. A small training set is inadequate in representing the complexity of the data and the model is more likely to overfit to the small training set. It is also more sensitive to noise. On the other hand, if the test size is too small, the evaluation metrics may not be as reliable there aren't enough predictions and the evaluation metric becomes too sensitive to the specific data points chosen for testing, resulting in more varied and less reliable estimations of model performance. We found a test_size value of around 0.5 to have a good balance between the two. 

#### Approach 2: Feature Engineering
Feature engineering involves transforming our input features into more relevant forms for our model.
One example of feature engineering that has already been implemented is the normalization of input features, which is automatically done by the model (our class instance) during training and prediction. Input features are normalized using z-normalization (through the private _normalize_z method in the class).<br>

Another method is to transform our features, changing their representation in a way to make them more suitable for multiple linear regression. Our class has a method called 'add_transformed_feature' which will transform a chosen feature. The transformed feature can either replace the old feature or be added as a new feature.<br>
*Note: a model should always be retrained after adding a new transformed feature.*
<br>
As can be seen in our previous scatter plots, the features 'World Freedom Index' and 'Cost of Living Index' seem to fit non-linearly to our target. In the cell below, we will carry out a polynomial transformation of these features (square our feature values) and have them replace their non-transformed versions.

In [ ]:
# Let's create a new model with the same hyperparameters as model2
model3 = MultipleLinearRegression()
model3.store_data(df, chosen_features, [target], random_state=100, test_size=0.5) # pass in seed to make data split the same

# Before training, let's transform our features
model3.add_transformed_feature("World Freedom Index", "WFI Square", np.square, replace=True)
model3.add_transformed_feature("Cost of Living Index", "CLI Square", np.square, replace=True)

# Train and validate model3
model3.train(target, alpha=0.01, epochs=600, show=False)
model3.validate(target)
pass

Unfortunately, our model has only improved very slightly. 

#### Approach 3: Locally Weighted Regression
Previously, we attempted to transform our features to have them fit linearly with the target. However, sometimes it is difficult to determine what transformations should be used. We attempt to sidestep this issue by using locally weighted regression instead. Locally weighted regression, also known as LOESS, is a non-parametric variation of multiple linear regression that has several benefits. Unlike normal multiple linear regression, LOESS makes no assumptions of global linearity and can flexibly capture non-linearity. LOESS is similar to normal linear regression except that instead of minimising mean square error, it instead minimises locally weighted mean square error. In short, training examples nearer to the point of prediction are weighted higher than those further away. This makes LOESS adaptive to local patterns and less sensitive to outliers.
<br>
However, LOESS isn't without its issues. Finding an appropriate value for the bandwidth parameter, which affects the spread of the gaussian kernel (the weight), is both important (due to the impact it has on model performance) and challenging. Furthermore, LOESS is more computationally intensive in that it has to compute the optimal parameter values for every prediction (as we have to minimise locally weighted cost). 
<br>
For this project, we have implemented LOESS as a subclass of our MultipleLinearRegression class. This allows us to inherit previously defined methods while making certain changes for LOESS. The main difference is that LOESS does not have a training step because optimal parameter values are obtained for each prediction (and hence the model does not store parameter values). See the class documentation below for more details.

In [ ]:
class LocallyWeightedRegression(MultipleLinearRegression):
    """Class for a locally weighted regression model.

    Extended Summary
    ----------------
    The model is a subclass of MultipleLinearRegression and inherits most
    of its functionality. Unlike in multiple linear regression, however, this
    model does not save any parameter values and are instead computed everytime
    the model makes a prediction (as it needs to fit to the locally weighted cost).
    Hence there is no training step. After training data is stored, the model can
    make predictions immediately.
    General user flow:
    1) Store data (expects pandas DataFrames) using store_data().
    2) Validate the model's performance using validate().

    Attributes
    ----------
    features, targets : list of str
        A list of strings containing the column names of the features and targets respectively.
        Initialised when data is stored in the instance using store_data().
    train_features, test_features, train_targets, test_targets : pandas.DataFrames
        Pandas Dataframes containing the features/targets for training/testing. Initialised
        when data is stored in the instance using store_data().

    Methods
    -------
    Inherited:
        store_data(data, feature_names, target_names, random_state=None, test_size=0.5)
            Takes the given data and separates the features and targets. Then splits the data
            for training and testing before storing it in the model's attributes.
        add_transformed_feature(feature_name, new_feature_name, transform_func, replace=False)
            Transforms a specified feature and adds it to the stored data.

    Overriden:
        train()
            This method is disabled. Parameter values are obtained for each
            prediction in locally weighted regression.
        predict(dfin_features, target, tau=1)
            Given some input data of features, the model returns its predictions for the specified target
            variable using locally weighted regression.
        validate(target, visualize=True, tau=1)
            Evaluate the model's ability to predict the specified target variable (its performance) using the
            test data. Will show visualizations of the predicted values against the true values for the test data
            and return various performance metrics.

    Notes
    -----
    Locally weighted regression is a non-parametric learning algorithm. The bandwidth parameter (tau) is very
    important value that has a significant impact on the model's performance.
    """

    def __init__(self):
        """Constructs a new instance of LocallyWeightedRegression.

        Notes
        -----
        Unlike the MultipleLinearRegression class, there is no need to store
        parameters nor the mean and standard deviation of the training features.
        In locally weighted regression, the parameter values are determined for
        each prediction.
        """

        self.features, self.targets = None, None # list of names
        self.train_features, self.train_targets = None, None
        self.test_features, self.test_targets = None, None

    def predict(self, dfin_features, target, tau=1):
        """Predict for the specified target with the given input features using
        locally weighted regression.

        Parameters
        ----------
        dfin_features : pandas.DataFrame
            Data containing input features to predict with, where each row is a set of input features (x).
            It is a DataFrame with shape (m,n), where m is the number of data points to be predicted
            and n is the number of feature variables.
        target : str
            Column name of target variable to be predicted.
        tau : float, default 1
            Bandwidth parameter value of the gaussian kernel (local weight).

        Returns
        -------
        y_hat : numpy.ndarray
            NumPy array that has a shape of (m,1) and contains the model's predictions for each
            of the input data points.

        Notes
        -----
        In locally weighted regression, the parameters for each prediction changes depending on the
        input features. Datapoints closer to the input feature values (point of prediction) are given
        more weightage than points further away. The bandwidth parameter (tau) determines the spread of
        the gaussian kernel (which determines the weightages of points). A smaller bandwidth parameter
        means a more concentrated gaussian kernel, making it more sensitive to neighbouring points as opposed
        to more distant ones.
        When computing the parameters using the matrix equation, this function finds the Moore-Penrose inverse
        rather than the regular inverse. This is because the square matrix in the equation (X_train.T @ W @ X_train)
        may be singular and therefore non-invertible. In this case, the Moore-Penrose inverse is sufficient for
        the purpose of optimising the locally-weighted cost function.
        """

        # Check that there is data stored for training and validation:
        if any(data is None for data in (self.train_features, self.train_targets, self.test_features, self.test_targets)):
            raise ValueError("Data missing. Run the store_data() method first.")
        # Check that valid target name is specified:
        elif target not in self.targets:
            raise ValueError("Specified target not found in stored data.")

        # Prepare training features and target:
        train_features_norm, means, stds = self._normalize_z(self.train_features)
        X_train = self._prepare_X(train_features_norm)
        y = self._prepare_y(self.train_targets.loc[:, [target]])

        # Normalize and prepare input features
        test_features_norm, _, _ = self._normalize_z(dfin_features, means, stds)
        X_in = self._prepare_X(test_features_norm)

        num_inputs = X_in.shape[0]
        y_hat = np.zeros(num_inputs) # for storing predictions

        # Make prediction for each input data (row in X_in):
        for i in range(num_inputs):
            # x is specific point at which you want to make prediction
            x = X_in[i,:] # (n,) 1D array for broadcasting purposes

            dist = np.sum((X_train - x)**2, axis=1)
            w = np.exp(-dist / (2 * tau**2)) # (m,)
            W = np.diag(w) # diagonal matrix (m,m)
            params = np.linalg.pinv(X_train.T @ W @ X_train) @ (X_train.T @ W @ y) # Moore-Penrose inverse
            pred = x.reshape(1,-1) @ params
            y_hat[i] = pred

        y_hat = y_hat.reshape(-1,1)
        return y_hat # (m,1)

    def validate(self, target, visualize=True, tau=1):
        """Evaluate the model's performance using testing data and return performance metrics.

        Extended Summary
        ----------------
        Using the testing data, the model will be evaluated by comparing its predictions (of the
        specified target variable) with the true target values. This comparison is visualised with
        scatter plots of the target variable against each input feature. The following metrics, which
        evaluate the multiple linear regression model's predictions, are computed and returned:
        - Mean Square Error
        - Mean Absolute Error
        - R2 coefficient of determination
        - Adjusted R2 coefficient of determination

        Parameters
        ----------
        target : str
            Column name of target variable to be predicted. The model will be evaluated based on its
            predictions for this target variable.
        visualize : float, default True
            If True, comparison between predictions and true values is visualized.
        tau : float, default 1
            Bandwidth parameter value of the gaussian kernel (local weight).

        Returns
        -------
        mse, mae, r2, r2_adj : float
            The numerical values of the model's performance metrics mentioned above.

        Notes
        -----
        - The R2 coefficient is usually a value between 0 and 1, in which 0 indicates that the model explains
        none of the variability and 1 indicates that the model explains all the variability. When there is more
        than 1 feature however, it is possible for R2 coefficient to be negative if the fit is particularly bad.
        - For multiple linear regression, the R2 coefficient is not a good performance metric, as increasing the
        number of features always increases the R2 value. The adjusted R2 value (r2_adj) is a metric that accounts
        for this "inflation" of the R2 value when the number of feature variables increases, and is therefore
        more appropriate for evaluating model fit.
        """

        self._target_valid(target)
        df_features = self.test_features
        df_targets = self.test_targets

        y_hat = self.predict(df_features, target, tau=tau)
        if visualize == True:
            self._visualize(df_features, df_targets, target, y_hat)
        y = self._prepare_y(df_targets.loc[:, [target]])
        mse, mae, r2, r2_adj = self._calc_metrics(y_hat, y)
        print(f"Mean Square Error: {mse:.5f} | Mean Absolute Error: {mae:.5f} | R2 Coefficient of Determination: {r2:.5f} | Adjusted R2: {r2_adj:.5f}")
        return mse, mae, r2, r2_adj


    def _target_valid(self, target):
        """Checks whether given target name is valid (found in stored data)"""

        if target not in self.targets:
            raise ValueError("Specified target not found in stored data.")


    @staticmethod
    def train():
        raise Exception("This method is non-functional for Locally Weighted Regression. Parameter values are computed during each prediction instead.")

In [ ]:
# Let's create a new model with the same hyperparameters as model2
loess_model = LocallyWeightedRegression()
loess_model.store_data(df, chosen_features, [target], random_state=100, test_size=0.5) # pass in seed to make data split the same

# Validate LOESS model
loess_model.validate(target, tau=3.5)
pass

Unfortunately, the performance of our model, while still decent, has decreased. Locally Weighted Regression is not the way to go for this project.

##### Extra demonstration of LOESS
However, just to demonstrate that locally weighted regression can be useful, this section will show the performance difference between a normal linear regression model and a LOESS model using housing data.

In [ ]:
# Multiple Linear Regression model
df_housing = pd.read_csv("housing_processed.csv")
housing_model_mlr = MultipleLinearRegression()
housing_model_mlr.store_data(df_housing, ["RM","DIS","INDUS"], ["MEDV"], random_state=100, test_size=0.5)
housing_model_mlr.train("MEDV", show=False)
housing_model_mlr.validate("MEDV")

In [ ]:
# Locally Weighted Regression Model
housing_model_loess = LocallyWeightedRegression()
housing_model_loess.store_data(df_housing, ["RM","DIS","INDUS"], ["MEDV"], random_state=100, test_size=0.5)
housing_model_loess.validate("MEDV")

The LOESS model has much better performance metric values. The effect that locally weighted regression has is very apparent in the scatterplot of MEDV against RM. In the normal multiple linear regression model, the predictions (in orange) follow a very strict line. In the locally weighted regression model however, the predictions are less linear and follow the true values more closely. 

### Discussion and Analysis

*Discuss your model and accuracy in solving the problem. Analyze the results of your metrics. Put any conclusion here.*
<br>
Out of all our models, model3 has the best performance:

In [ ]:
model3.validate(target)
pass

As seen above, our model has a fairly low mean square error and mean absolute error which means our model is decently accurate in its predictions. Meanwhile, the model's adjusted R2 coefficient is 0.56045, which suggests that it explains slightly more than half of the variance in the dependent variable (target). This means that the model has moderate explanatory power but there is much room for improvement.
<br>
The biggest limitation of this model is the lack of data points. Intuitively, more data results in a better representation of the population. The low data count also made our model particularly susceptible to high variability in performance due to the splitting of data. For example, the randomly chosen datapoints for the training set may not form a good representation of the overall population and may result in the model learning less than ideal parameter values. Again, this issue is mitigated with more data points. We also suspect that the poor results of the locally weighted regression model might be similarly due to this lack of data, as it performed much better with the housing dataset (which has a lot more values). 
<br> This project also emphasised the importance of feature engineering as it can be seen that many of the variables do not fit linearly with the target variable (hence why we attempted to use locally weighted regression). With that all in mind, we identified some possible future improvements:
1) Obtain more data points, possibly ignore variables that limit the number of data points available
2) More thorough data cleaning; remove outliers and noise (as multiple linear regression is particularly susceptible to noise due to using MSE as a cost function)
3) Introduce regularization parameters to combat overfitting (especially if we introduce many polynomial features)
4) More stringent feature selection: we chose our selection by looking at the r2 coefficient between a particular feature and the target, but never considered looking at the correlation between the target and transformed versions of the feature. We did not do this due to time constraints as there are many transformations that can be applied and many possible feature variables to look at.